In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cc-labelled/cc_data_cleaned_labelled2.csv
/kaggle/input/cc-labelled/cc_data_cleaned_labelled.csv
/kaggle/input/cc-data-2025-04-2025-10-20/pcm_report_2025-02-01_2025-04-19.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-03-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-09-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-05-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-04-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-02-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/pcm_report_2025-04-20_2025-10-20.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-08-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-06-01.csv
/kaggle/input/cc-data-2025-04-2025-10-20/credit-card-statement-transactions-2025-07-01.csv


In [66]:
import pandas as pd
import glob

# Example: load all CSVs in a folder
files = glob.glob("/kaggle/input/cc-data-2025-04-2025-10-20/*.csv")
dfs = [pd.read_csv(f) for f in files]

# Concatenate and reset index
df = pd.concat(dfs, ignore_index=True)

# Standardize column names
df.columns = df.columns.str.lower().str.strip()

# Example cleanup
df['date'] = pd.to_datetime(df['date'])
df['amount'] = df['amount'].astype(float)


In [67]:
standard_cols = ["date", "description", "type", "amount", "card", "category"]

In [68]:
data = df.copy()

data["date"] = data["date"].combine_first(data["transaction_date"])
data["details"] = data["details"].combine_first(data["description"])

In [69]:
# Combining duplicate type columns
# Get all columns that share the name "type"
type_cols = [i for i, c in enumerate(data.columns) if c == "type"]

# Extract them
df_type_1 = data.iloc[:, type_cols[0]]
df_type_2 = data.iloc[:, type_cols[1]]

data['type_combined'] = df_type_1.combine_first(df_type_2)

data = data.drop(data.columns[type_cols], axis=1)

data = data.rename(columns={'type_combined': 'type'})

In [70]:
# Combining duplicate amount columns
# Get all columns that share the name "amount"
amount_cols = [i for i, c in enumerate(data.columns) if c == "amount"]
print(amount_cols)

# Extract them
df_amount_1 = data.iloc[:, amount_cols[0]]
df_amount_2 = data.iloc[:, amount_cols[1]]

data['amount_combined'] = df_amount_1.combine_first(df_amount_2)

data = data.drop(data.columns[amount_cols], axis=1)

data = data.rename(columns={'amount_combined': 'amount'})

[4, 8]


In [71]:
data = data.drop(["transaction_date", "post_date", "card holder name", "time", "description"],axis=1)

In [72]:
# Creating purchases from only the purchase transactons (exclude payments, refunds, cash withdrawal, etc.)
purchases = data.loc[(data["type"] == "Purchase") | (data["type"] == "PURCHASE")]

#ensure all are positive values (PC and WS handle transactions vs payments differently)
purchases["amount"] = np.abs(purchases["amount"])


/tmp/ipykernel_37/1928999744.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [73]:
import re

# Helper function to clean up transaction descriptions by removing symbols/puncuation and remove excess whitespace

def clean_description(desc):
    desc = desc.upper()
    desc = re.sub(r'[^A-Z0-9 ]', '', desc)  # remove punctuation
    desc = re.sub(r'\s+', ' ', desc).strip()
    return desc

In [74]:
import numpy as np

# Defining general categories and keywords for categorize common transactions

category_keywords = {
    'DINING OUT': ['BAR', 'RESTAURANT', 'CAFE', 'TIM HORTONS', 'STARBUCKS', 'EUREST CSA', 'MCDONALDS', 'CHIPOTLE', 'LOCAL LIBERTY'],
    'GROCERIES': ['NOFRILLS', 'NO FRILLS', 'JORDANS NF','LONGOS','WALMART', 'COSTCO', 'LOBLAWS', 'METRO', 'GROCERY', 'FOODLAND', 'FRESHCO'],
    'GAS': ['SHELL', 'PETRO', 'ESSO', 'GAS'],
    'SUBSCRIPTIONS': ['NETFLIX', 'PRIME MEMBER', 'PROTON', 'PRIME', 'PRIMEVIDEO', 'PATREON', 'APPLECOM', 'AUDIBLE'],
    'UBER RIDES' : ['UBER TRIP', 'UBERTRIP'],
    'PUBLIC TRANSIT' : ['PRESTO'],
    'UBER EATS' : ['UBER EATS', 'UBEREATS'],
    'ALCOHOL' : ['LCBO', 'WINE RACK'],
    'AMAZON' : ['AMAZON', 'AMZN'],
    'SHOPPERS/REXALL' : ['SHOPPERS DRUG', 'REXALL']
}

# Categorization function
# This takes a transaction description as an input and checks if it contains any of the keywords for a category.
# It will categorize the transaction based on the first keyword found and label it as 'Other' if no match is found.
def categorize(transaction_description):
    for category, keywords in category_keywords.items():
        if any(key in transaction_description for key in keywords):
            return category
    return 'Other'

In [75]:
# print(purchases.dtypes)

# Trying to ensure transaction description is a string for processing.
purchases["details"] = purchases["details"].astype(str)

# print(purchases.dtypes)

/tmp/ipykernel_37/1848363558.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
# Cleaning the transaction descriptions
purchases['clean_desc'] = purchases['details'].apply(clean_description)

# Replacing the current transaction descriptions with the cleaned versions.

purchases = purchases.drop("details", axis=1)
purchases = purchases.rename(columns={'clean_desc': 'details'})
# print(purchases)

/tmp/ipykernel_37/383496451.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
# purchases.to_csv("/kaggle/working/test_nocat.csv")

In [78]:
# Categorizing the transactions.
purchases["category"] = purchases['details'].apply(categorize)

print(purchases)

# This shows the distribution of categories. Need to reduce the amount of 'Other' category as much as possible.
print(purchases["category"].value_counts())

          date      type  amount                  details         category
1   2025-04-17  PURCHASE   40.67             APPLECOMBILL    SUBSCRIPTIONS
2   2025-04-15  PURCHASE   44.06           STEAM PURCHASE            Other
3   2025-04-14  PURCHASE   33.56    JORDANS NF TORONTO KI        GROCERIES
4   2025-04-14  PURCHASE   43.44         MARTINGROVE ESSO              GAS
5   2025-04-13  PURCHASE   62.77    JORDANS NF TORONTO KI        GROCERIES
..         ...       ...     ...                      ...              ...
972 2025-07-23  Purchase   28.25  SQ KINGSWAY BOXING CLUB            Other
974 2025-07-23  Purchase    7.54         EUREST CSA 23173       DINING OUT
975 2025-07-23  Purchase   12.49      PROTON AG PROTON AG    SUBSCRIPTIONS
977 2025-07-23  Purchase    7.66     REXALL PHARMACY 8174  SHOPPERS/REXALL
978 2025-07-24  Purchase    7.54         EUREST CSA 23173       DINING OUT

[716 rows x 5 columns]
category
Other              206
GROCERIES          109
DINING OUT          9

In [79]:
# Exporting the labelled data

purchases.to_csv("/kaggle/working/test.csv")

In [80]:
# Reimporting the labelled data with some extra cleaning and manual labelling done.

df = pd.read_csv("/kaggle/input/cc-labelled/cc_data_cleaned_labelled2.csv")
print(df)

     Unnamed: 0       date      type  amount                  details  \
0             1  4/17/2025  PURCHASE   40.67             APPLECOMBILL   
1             2  4/15/2025  PURCHASE   44.06           STEAM PURCHASE   
2             3  4/14/2025  PURCHASE   33.56    JORDANS NF TORONTO KI   
3             4  4/14/2025  PURCHASE   43.44         MARTINGROVE ESSO   
4             5  4/13/2025  PURCHASE   62.77    JORDANS NF TORONTO KI   
..          ...        ...       ...     ...                      ...   
711         972  7/23/2025  Purchase   28.25  SQ KINGSWAY BOXING CLUB   
712         974  7/23/2025  Purchase    7.54         EUREST CSA 23173   
713         975  7/23/2025  Purchase   12.49      PROTON AG PROTON AG   
714         977  7/23/2025  Purchase    7.66     REXALL PHARMACY 8174   
715         978  7/24/2025  Purchase    7.54         EUREST CSA 23173   

            category  
0      SUBSCRIPTIONS  
1              Other  
2          GROCERIES  
3                GAS  
4       

In [81]:
# Convert to datetime if not already
df["date"] = pd.to_datetime(df["date"])

# Create a "month" column in YYYY-MM format
df["month"] = df["date"].dt.to_period("M").astype(str)

In [82]:
# Organizing transactions by month for monthly breakdowns.

df = df[df["category"] != "Other"]

monthly_totals = (
    df.groupby(["month", "category"])["amount"]
      .sum()
      .reset_index()
)

In [83]:
avg_monthly_spending = (
    monthly_totals.groupby("category")["amount"]
    .mean()
    .reset_index()
    .sort_values("amount", ascending=False)
)

print(avg_monthly_spending)

          category      amount
2       DINING OUT  551.209000
4        GROCERIES  412.035556
8        UBER EATS  214.787143
3              GAS  201.698889
1           AMAZON  154.011250
7    SUBSCRIPTIONS  128.114444
0          ALCOHOL  117.710000
6  SHOPPERS/REXALL  109.591111
9       UBER RIDES   84.052500
5   PUBLIC TRANSIT   15.970000


In [84]:
# Getting default colors for Plotly so I can manually set the colors and make all charts consistent.

# default_colors = px.colors.qualitative.Plotly
# print(default_colors)

In [85]:
import plotly.express as px


color_map = {'DINING OUT': '#00CC96',
            'GROCERIES': '#636EFA',
            'GAS': '#EF553B',
            'SUBSCRIPTIONS': '#AB63FA',
            'UBER RIDES' : '#FFA15A',
            'PUBLIC TRANSIT' : '#19D3F3',
            'UBER EATS': '#FF6692',
            'ALCOHOL': '#B6E880',
            'AMAZON': '#FF97FF',
            'SHOPPERS/REXALL': '#FECB52',
            }

fig = px.bar(
    monthly_totals,
    x="month",
    y="amount",
    color="category",
    color_discrete_map = color_map,
    title="Monthly Spending by Category",
    barmode="stack",   # or "stack" for stacked bars
    text_auto=True
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Total Spending ($)",
    legend_title="Category",
    template="plotly_white"
)

fig.show()

fig2 = px.bar(
    avg_monthly_spending,
    x="category",
    y="amount",
    color="category",
    color_discrete_map = color_map,
    title="Average Monthly Spending by Category",
    text_auto=True
)

fig2.update_layout(
    xaxis_title="Month",
    yaxis_title="Total Spending ($)",
    legend_title="Category",
    template="plotly_white"
)

fig2.show()